In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")


/modules/apps/miniconda/4.8.3/envs/jupyterhub-stable/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [02:52<00:00, 86.21s/it] 


# Few Shot Training

In [3]:
#import pandas as pd
#data = pd.read_csv('Dataset_696.csv')

#prompt = data['Prompt']
#output = data['Output']
#prompt_question = data['Question'].iloc[0:3]



FileNotFoundError: [Errno 2] No such file or directory: '/content/Dataset_696.csv'

In [ ]:
#text = ""
#for i in range(len(prompt)):
#  sample = prompt.iloc[i]+" => "+output.iloc[i]+"."
#  text += sample

#print("Training Prompt : ")
#print(text)

In [ ]:
#for inp in prompt_question:
#   new_text = text + "Please learn response like above and generate response for the below."
#   new_text += inp+ " => "
#   inputs = tokenizer(new_text, return_tensors="pt")
#   outputs = model.generate(**inputs)
#   print("Prompt: ",inp)
#   print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
#   print("-----------------------\n")

# One Shot Training

In [ ]:
#text = """
#Go to Setup > Company > General Preferences > click the Languages subtab > In the Language column, select the language > Click Add > Click Save => click[Setup], click[Company], click[General Preferences], click[English], click[Language], click[Add], click[Save]. 

#Please provide answer like above Prompt for below Prompt :
#Go to Setup > Payroll > Setup Tasks > Set Up Payroll > From the Subsidiary list, select a subsidiary => 
#"""
#inputs = tokenizer(text, return_tensors="pt")
#outputs = model.generate(**inputs)
#print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


# Zero Shot Training

In [ ]:

#text = """
#Please generate response for the below.
#Go to Setup > Payroll > List > Click Void. => 
#"""
#inputs = tokenizer(text, return_tensors="pt")
#outputs = model.generate(**inputs)
#print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


### Creating function to select best prompt

In [6]:
import pandas as pd
data2 = pd.read_csv('696_Dataset_2.csv')

new_prompt = data2['Prompt']
new_output = data2['Output']
new_prompt_question = data2['Questions']
new_expected_output = data2['Expected Output']

In [7]:
!pip3 install textdistance

import textdistance
from collections import defaultdict

def calculateSimilarityScore(ques_prompt, inp_prompt):
  return textdistance.sorensen(ques_prompt, inp_prompt)

def chooseBestPrompt(_prompt):
   scores = [[calculateSimilarityScore(_prompt, _input_prompt), _input_prompt, _new_output] for _input_prompt,_new_output in zip(new_prompt,new_output) ]
   scores.sort(key = lambda x:x[0], reverse = True)
   return [scores[0][1], scores[0][2]]


best_prompt = defaultdict(list)

for _prompt in new_prompt_question:
    best_prompt[_prompt] = chooseBestPrompt(_prompt)
    
    
for k,v in best_prompt.items():
  print("For question prompt : ")
  print(k)
  print("The best possible input prompt is : ")
  print(v[0])
  print("----------------------------------------------\n\n")


Defaulting to user installation because normal site-packages is not writeable
For question prompt : 
1. Go to Transactions > Bank > Make Deposits > List. 2. From the Deposits list, click Edit next to the deposit #1 you want to modify. 3. On the Deposit page, modify the saved deposit as required. 4. Click Save
The best possible input prompt is : 
1. Go to Transactions > Bank > Make Deposits > List. 2. From the Deposits list, click Edit next to the deposit you want to delete. 3. On the Deposit page, under Actions, click Delete. 4. click OK
----------------------------------------------


For question prompt : 
1. Go to Setup > Company > Enable Features.
2. On the Company subtab, check the Multiple Currencies box.
3. Click Save.
The best possible input prompt is : 
1. Go to Setup > Company > Enable Features.
2. On the Company subtab, scroll to the International features.
3. Check the Multi-Language feature box.
4. Click Save.
----------------------------------------------


For question p

## Creating function for evaluation matrix based of String matching

In [8]:
!pip3 install textdistance

import textdistance
from collections import defaultdict
import re

def metricUsingStringMatching(expected, original):
   _expected = expected.split("], ")
   _original = original.split("], ")
   n = len(_original)
   counts = 0
   for exp in _expected:
     for org in _original:
       if(textdistance.hamming.normalized_similarity(exp,org)>=0.98):
          counts += 1
   return counts/n

best_prompt = defaultdict(float)

for exptd in new_expected_output:
   for out in new_output:
      print("Given Ground Truth = ")
      print(exptd)
      print("and current Output = ")
      print(out)
      print("Accuracy score = ", metricUsingStringMatching(exptd, out))
      print("----------------------------------------------\n\n")



Defaulting to user installation because normal site-packages is not writeable
Given Ground Truth = 
1. click[Transactions], click[Bank], click[Make Deposits], click[List], 2. search[Deposit #1], select[Deposit #1], click[Edit], 3. modify[Deposit #1], 4. click[Save]
and current Output = 
1. click[Transactions], click[Bank], click[Make Deposits], click[List], 2. search[Deposit #1], select[Deposit #1], click[Edit], 3. click[Delete], 4. click[OK]
Accuracy score =  0.7777777777777778
----------------------------------------------


Given Ground Truth = 
1. click[Transactions], click[Bank], click[Make Deposits], click[List], 2. search[Deposit #1], select[Deposit #1], click[Edit], 3. modify[Deposit #1], 4. click[Save]
and current Output = 
1. click[Setup], click[Company], click[General Preferences], 2. click[Language], 3. click[English], 4. click[Add], 5. click[Save]
Accuracy score =  0.0
----------------------------------------------


Given Ground Truth = 
1. click[Transactions], click[Bank

## Testing One Shot with above implementation

In [21]:
def chooseBestPrompt(_prompt):
   scores = [[calculateSimilarityScore(_prompt, _input_prompt), _input_prompt, _new_output] for _input_prompt,_new_output in zip(new_prompt,new_output) ]
   scores.sort(key = lambda x:x[0], reverse = True)
   return [scores[0][1], scores[0][2]]

In [22]:
def one_shot(test_question_prompt, ground_truth, test_input_prompt, test_input_output):
    for _prompt, _output in zip(test_input_prompt, test_input_output):
        text = """
        {p} => {o}
        Please learn response like above and generate response for the below :
        """.format(p=_prompt,o=_output)
        
        text += test_question_prompt + " => "
        inputs = tokenizer(text, return_tensors="pt", padding = True)
        outputs = model.generate(**inputs, max_new_tokens=500)
        print(text)
        generated_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        print(generated_output)
        print("Accuracy Score = ",metricUsingStringMatching(generated_output[0], ground_truth))
        print("--------------------------------------------------------------------------------------------")

In [23]:
def one_shot_with_selected_prompt(test_question_prompt, ground_truth):
    inputs = chooseBestPrompt(test_question_prompt)
    _prompt, _output = inputs[0], inputs[1]
    text = """
    {p} => {o}
    Please learn response like above and generate response for the below :
    """.format(p=_prompt,o=_output)

    text += test_question_prompt + " => "
    inputs = tokenizer(text, return_tensors="pt", padding = True)
    outputs = model.generate(**inputs, max_new_tokens=500)
    print(text)
    generated_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(generated_output)
    print("Accuracy Score = ",metricUsingStringMatching(generated_output[0], ground_truth))
    print("********************************************************************************************")
    
test_input_prompt = data2['Prompt'].iloc[0:4]
test_input_output = data2['Output'].iloc[0:4]
test_question_prompt = data2['Questions'].iloc[0:3]
test_question_output = data2['Expected Output'].iloc[0:3]

for p,o in zip(test_question_prompt, test_question_output):
    print("Normal One shot = ")
    one_shot(p, o, test_input_prompt, test_input_output)
    print("\n")
    
    print("With Selected Prompt")
    one_shot_with_selected_prompt(p,o)
    print("xxxxxxxxxxxxxxxx New Prompt xxxxxxxxxxxxxxxxxxxxxxxxxxx\n")

Normal One shot = 

        1. Go to Transactions > Bank > Make Deposits > List. 2. From the Deposits list, click Edit next to the deposit you want to delete. 3. On the Deposit page, under Actions, click Delete. 4. click OK => 1. click[Transactions], click[Bank], click[Make Deposits], click[List], 2. search[Deposit #1], select[Deposit #1], click[Edit], 3. click[Delete], 4. click[OK]
        Please learn response like above and generate response for the below :
        1. Go to Transactions > Bank > Make Deposits > List. 2. From the Deposits list, click Edit next to the deposit #1 you want to modify. 3. On the Deposit page, modify the saved deposit as required. 4. Click Save => 
['1. click[Transactions], click[Bank], click[Make Deposits], click[List], 2. search[Deposit #1], select[Deposit #1], click[Edit], 3. click[Save], 4. click[Save]']
Accuracy Score =  0.8888888888888888
--------------------------------------------------------------------------------------------

        1. Go to Se